<a href="https://colab.research.google.com/github/omarbecerrasierra/MLOpsChallenge/blob/main/Caso2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de dependencias necesarias
!pip install -U langchain-core
!pip install -U pymupdf
!pip install -U langchain
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 2.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is i

In [ ]:
import fitz  # PyMuPDF
from langchain import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm.notebook import tqdm
import os
import json
import re
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('API_KEY')

In [ ]:
# Función para limpiar el texto del CV
def clean_text(text):
    text = text.lower()
    replacements = {
        r'\n': ' ', r'´ o': 'o', r'´ ı': 'i', r'´ a': 'a',
        r'´ u': 'u', r'´ e': 'e', r'˜ n': 'n', r' %': '%',
        r'\x83': '', '´ i̇':'i', 'ï':''
    }
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Función para extraer texto de un PDF
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

In [ ]:
class Info(BaseModel):
    full_name: str = Field(description="Full name of the candidate in lower case.")
    contact: str = Field(description="Contact information (email in format lower_case@email.com or phone number in format +(country code) phone number).")
    years_experience: int = Field(description="Total years of professional experience as a number")
    ai_education: str = Field(description="Whether the candidate has education in artificial intelligence. (Options: 'Yes' or 'No')")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Definición del template de prompt
template = """
You will be provided with a block of text extracted from a CV.
Your task is to extract specific information and output it in a JSON format.
Only include the requested fields in the JSON object, and ensure there is no additional text.
Text from CV: {cv_text}
Output Format: {format_instructions}
"""
parser = JsonOutputParser(pydantic_object=Info)

prompt = PromptTemplate(
    template=template,
    input_variables=["cv_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
# Crear el prompt y la cadena LLM
chain = prompt | llm | parser

In [ ]:
# Función para procesar una carpeta de archivos PDF
def process_cv_folder(folder_path):
    results = []

    # Iterar sobre todos los archivos PDF en la carpeta
    for file_name in tqdm(os.listdir(folder_path)):
        if file_name.endswith('.pdf'):
            try:
                file_path = os.path.join(folder_path, file_name)

                # Extracción y limpieza del texto del CV
                extracted_text = extract_text_from_pdf(file_path)
                cleaned_text = clean_text(extracted_text)

                # Ejecutar la cadena para extraer la información del CV
                result = chain.invoke({"cv_text": cleaned_text})

                # Almacenar el resultado junto con el nombre del archivo
                results.append({"file_name": file_name, "extracted_data": result})
            except Exception as e:
                print('Error', e)
    return results

# Ruta de la carpeta que contiene las hojas de vida
folder_path = "dataset/cvs"

# Procesar todas las hojas de vida en la carpeta
cv_results = process_cv_folder(folder_path)

# Mostrar los resultados en un formato JSON legible
for cv in cv_results:
    print(f"File Name = {cv['file_name']}:")
    print("Info:")
    print(json.dumps(cv['extracted_data'], indent=4))

  0%|          | 0/9 [00:00<?, ?it/s]

File Name = CV8.pdf:
Info:
{
    "full_name": "dr.santosh kakade",
    "contact": "drsantoshkakade@gmail.com",
    "years_experience": 20,
    "ai_education": "No"
}
File Name = CV4.pdf:
Info:
{
    "full_name": "dyah hediyati s.kom",
    "contact": "dyahhediyati@gmail.com",
    "years_experience": 4,
    "ai_education": "No"
}
File Name = CV7.pdf:
Info:
{
    "full_name": "ringgi cahyo dwiputra",
    "contact": "ringgicahyo@gmail.com",
    "years_experience": 3,
    "ai_education": "No"
}
File Name = CV9.pdf:
Info:
{
    "full_name": "loren shevitz",
    "contact": "loren@shevitz.org",
    "years_experience": 21,
    "ai_education": "No"
}
File Name = CV2.pdf:
Info:
{
    "full_name": "powell finwood",
    "contact": "hello@reallygreatsite.com",
    "years_experience": 4,
    "ai_education": "No"
}
File Name = CV5.pdf:
Info:
{
    "full_name": "immanuel abraham mahardhika yudantoro tobing",
    "contact": "dhikayudano@gmail.com",
    "years_experience": 6,
    "ai_education": "No"
}
F